<a href="https://colab.research.google.com/github/fitocuan/CE888_2020/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [104]:
df = pd.read_csv('jester-data-1.csv')
print(df.head())
print(df.shape)

    74  -7.82   8.79  -9.66  -8.16  ...  99.22  -5.63  99.23  99.24  99.25
0  100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
1   49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
2   48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3   91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
4  100  -6.17  -3.54   0.44  -8.50  ...  -3.35   0.05  -9.08  -5.05  -3.45

[5 rows x 101 columns]
(24982, 101)


In [105]:

arr = df.drop(df.columns[0], axis=1).values
print(arr)

[[ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.    8.35 99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:

arr_new, idx = replace(arr, 0.1)

In [108]:

rated = np.where(arr!=99)
rated_new = np.where(arr_new!=99)
print(len(rated[0]), rated[1].shape)




1810381 (1810381,)


In [109]:
n_latent_factors = 2

user_ratings = arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

print(user_ratings)


[[ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.    8.35 99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]


In [0]:

def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating, alpha=0.01):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]

    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)


In [111]:
sgd() 

Streaming output truncated to the last 5000 lines.
[-0.094 -0.08 ]
[-0.027 -0.056]
[0.371 0.417]
[-0.254 -0.248]
[-0.132 -0.215]
[0.131 0.3  ]
[-0.171 -0.174]
[-0.012 -0.068]
[ 0.197 -0.664]
[-0.06  -0.081]
[0.079 0.06 ]
[0.017 0.018]
[0.061 0.05 ]
[0.17 0.09]
[-0.028 -0.011]
[ 0.002 -0.192]
[-0.002 -0.019]
[1.04  0.376]
[-0.527 -0.401]
[-0.023 -0.105]
[-0.08 -0.06]
[-0.153  0.222]
[-0.133  0.103]
[-0.04   0.239]
[-0.014  0.088]
[-0.099 -0.065]
[0.026 0.017]
[-0.044 -0.22 ]
[-0.05   0.205]
[0.048 0.024]
[0.052 0.041]
[-0.021  0.054]
[-0.105 -0.075]
[-0.029 -0.017]
[-0.016 -0.263]
[-0.027 -0.138]
[0.02  0.012]
[-0.024 -0.078]
[-0.061  0.112]
[-0.038  0.147]
[-0.03  -0.032]
[-0.021 -0.068]
[-0.019 -0.022]
[0.107 0.463]
[-0.077 -0.137]
[-0.033 -0.026]
[-0.056 -0.094]
[0.354 0.228]
[-0.072 -0.14 ]
[0.29  0.381]
[-0.025 -0.234]
[-0.145 -0.126]
[-0.114 -0.086]
[-0.071 -0.014]
[-0.048 -0.138]
[-0.043 -0.046]
[-0.07  -0.117]
[-0.016 -0.011]
[0.137 0.386]
[0.134 0.272]
[-0.091 -0.112]
[-0.077 -

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in multiply
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in subtract


Streaming output truncated to the last 5000 lines.
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]

KeyboardInterrupt: ignored

In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)

values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data